In [ ]:
import pandas as pd
import statsmodels.formula.api as smf
from scipy.stats import f_oneway
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import papermill as pm

In [ ]:
# TODO: note that comparison_type must be set by `launch_analyze_deeptools_results.ipynb`
comparison_type = None

In [45]:
print(comparison_type)
matrix_path = f"output/{comparison_type}/DMSO_3MB.computeMatrix.vals.mat.tab"

### Collect data on the height of each peak contributing to a profile

`DMSO_3MB.computeMatrix.vals.mat.tab` contains a matrix with the following format:
- rows are individual peaks
- columns are one of the 600 points in the plot for each replicate
- max height of an individual peak is then the max in that row for the 600 columns related to the replicate

In [12]:
matrix = pd.read_csv(matrix_path, sep="\t", skiprows=2)
fixed_columns = matrix.columns[1:]
matrix = matrix.iloc[:, :-1]
matrix.columns = fixed_columns

# peak_height is rows: replicate, columns: individual region's max value (peak height)
matrix_adj = matrix.copy()
matrix_adj.columns = matrix.columns.str.split(".").str[0]
peak_height = matrix_adj.T.groupby(matrix_adj.columns).max()

In [44]:
# Get the max plotted peak height (average across regions) for each replicate
# to constrain the yMax for plot heatmap
location_heights = matrix.mean(axis=0)
display(location_heights.max())

0.09237984849057518

### Violin plot

In [ ]:
peak_height_formatted = peak_height.T.melt(var_name="condition_replicate", value_name="peak_height")
peak_height_formatted[['condition', 'replicate']] = peak_height_formatted['condition_replicate'].str.split("_", expand=True)
# peak_height_formatted = peak_height_formatted.drop(columns="condition_replicate")

# Create the violin plot
plt.figure(figsize=(10, 6))
sns.violinplot(x='condition_replicate', y='peak_height', hue='condition', data=peak_height_formatted)

plt.title(f"Violin Plot Grouped by Condition and Replicate ({comparison_type})")
plt.xlabel("Condition and Replicate")
plt.ylabel("Peak height")
plt.xticks(rotation=45) 
plt.tight_layout()
plt.show()
# Because of the directory this is launched to, this will actually be in "output/{comparison_type}"
plt.savefig(f"peak_height_violin_plot.svg")
plt.close()

# Plot showing only up to the third quartile of each row
max_third_quartile = peak_height.quantile(0.75, axis=1).max()
y_max = max_third_quartile + 0.5 * max_third_quartile
plt.figure(figsize=(10, 6))
plt.ylim(0, y_max)
sns.violinplot(x='condition_replicate', y='peak_height', hue='condition', data=peak_height_formatted)
plt.title(f"Violin Plot Grouped by Condition and Replicate ({comparison_type})")
plt.xlabel("Condition and Replicate")
plt.ylabel("Peak height")
plt.xticks(rotation=45) 
plt.tight_layout()
plt.show()
# Because of the directory this is launched to, this will actually be in "output/{comparison_type}"
plt.savefig(f"peak_height_violin_plot_zoomed.svg")


### Mixed-effects model

The P > |z| value for the condition[T.DMSO] is the p-value for there being a difference in peak height between conditions 3MB and DMSO.

In [ ]:
peak_height_formatted = peak_height.T.melt(var_name="condition_replicate", value_name="peak_height")
peak_height_formatted[['condition', 'replicate']] = peak_height_formatted['condition_replicate'].str.split("_", expand=True)
peak_height_formatted = peak_height_formatted.drop(columns="condition_replicate")

# Fit a mixed-effects model
model = smf.mixedlm("peak_height ~ condition", peak_height_formatted, groups=peak_height_formatted["replicate"])
result = model.fit()
print(result.summary())

### ANOVA test

In [ ]:
peak_height_means = peak_height.mean(axis=1)

condition_means_3MB = peak_height_means.loc[peak_height_means.index.str.contains("3MB")]
condition_means_DMSO = peak_height_means.loc[peak_height_means.index.str.contains("DMSO")]

# Perform a one-way ANOVA
f_stat, p_value = f_oneway(condition_means_DMSO, condition_means_3MB)

print(f"F-statistic: {f_stat}, p-value: {p_value}")